# Import libs

In [23]:
# Common libs
import numpy as np
import matplotlib.pyplot as plt

# Tensorflow
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

print('Import succesfully')

Import succesfully


# Prepare dataset

Download mnist dataset. The dataset contains 60,000 images for the training and 10,000 images for the testing. Then normalize the images from range [0, 255] to [0, 1] and resize to 28 x 28 x 1.

In [24]:
num_classes = 10
img_rows, img_cols, img_channels = 28, 28, 1
input_shape = (img_rows, img_cols, img_channels)

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [25]:
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train = X_train.reshape(X_train.shape[0], *input_shape)
X_test = X_test.reshape(X_test.shape[0], *input_shape)

# Build and train LeNet5

In [26]:
class LeNet5(Model):

  def __init__(self, num_classes):
    """Create the model and its layer.
    :param num_classes: Number of classes to predict from.
    """

    super(LeNet5, self).__init__()
    self.conv1 = Conv2D(6, kernel_size=(5, 5), padding='same', activation='relu')
    self.conv2 = Conv2D(16, (5, 5), activation='relu')
    self.max_pool = MaxPooling2D(pool_size=(2, 2))
    self.flatten = Flatten()
    self.dense1 = Dense(120, activation='relu')
    self.dense2 = Dense(84, activation='relu')
    self.dense3 = Dense(num_classes, activation='softmax')

  def call(self, inputs):
    """ Apply the layers in order to process the input.
    :param x: Input tensor
    :return: Output tesor
    """

    x = self.max_pool(self.conv1(inputs))  # 1st block
    x = self.max_pool(self.conv2(x))  # 2nd block
    x = self.flatten(x)
    x = self.dense3(self.dense2(self.dense1(x)))  # dense layers
    return x

# Classifying MNIST

`sparse_categorical_crossentropy` performs the same as `categorical_crossentropy`, but the former directly takes the ground-truth labels as inputs instead of one one-hot encoded label. Used `sparse_categorical_crossentropy` saves us from manually having to transform the labels.

In [29]:
model = LeNet5(num_classes)

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

callbacks = [EarlyStopping(patience=3, monitor='val_loss'),
             TensorBoard(log_dir='./logs', histogram_freq=1)]

In [31]:
batched_input_shape = tf.TensorShape((None, *input_shape))
model.build(input_shape=batched_input_shape)

In [32]:
model.fit(X_train, y_train, batch_size=32, 
          epochs=80, validation_data=(X_test, y_test),
          verbose=2, callbacks=callbacks)

Epoch 1/80
1875/1875 - 5s - loss: 0.5336 - accuracy: 0.8319 - val_loss: 0.1463 - val_accuracy: 0.9563
Epoch 2/80
1875/1875 - 4s - loss: 0.1314 - accuracy: 0.9599 - val_loss: 0.0826 - val_accuracy: 0.9741
Epoch 3/80
1875/1875 - 4s - loss: 0.0903 - accuracy: 0.9725 - val_loss: 0.0710 - val_accuracy: 0.9755
Epoch 4/80
1875/1875 - 4s - loss: 0.0732 - accuracy: 0.9770 - val_loss: 0.0596 - val_accuracy: 0.9798
Epoch 5/80
1875/1875 - 4s - loss: 0.0625 - accuracy: 0.9806 - val_loss: 0.0536 - val_accuracy: 0.9811
Epoch 6/80
1875/1875 - 4s - loss: 0.0549 - accuracy: 0.9827 - val_loss: 0.0540 - val_accuracy: 0.9814
Epoch 7/80
1875/1875 - 4s - loss: 0.0482 - accuracy: 0.9851 - val_loss: 0.0439 - val_accuracy: 0.9863
Epoch 8/80
1875/1875 - 4s - loss: 0.0443 - accuracy: 0.9860 - val_loss: 0.0467 - val_accuracy: 0.9841
Epoch 9/80
1875/1875 - 4s - loss: 0.0405 - accuracy: 0.9872 - val_loss: 0.0378 - val_accuracy: 0.9873
Epoch 10/80
1875/1875 - 4s - loss: 0.0367 - accuracy: 0.9884 - val_loss: 0.0397 - 